In [0]:
from pyspark.sql import functions as F

try:
    df_bronze = spark.table("medalhao.bronze.ft_consumidores")

    df_deduplicated = df_bronze.dropDuplicates(["customer_id"])

    df_silver = df_deduplicated.select(
        F.col("customer_id").alias("id_consumidor"),
        F.col("customer_zip_code_prefix").alias("prefixo_cep"),
        
        F.upper(F.col("customer_city")).alias("cidade"),
        F.upper(F.col("customer_state")).alias("estado")
    )

    df_silver.write.mode("overwrite").saveAsTable("medalhao.silver.ft_consumidores")
    
    print("Tabela medalhao.silver.ft_consumidores criada.")

except Exception as e:
    print(f"Erro ao processar silver.ft_consumidores: {e}")

In [0]:
from pyspark.sql import functions as F

try:
    df_bronze = spark.table("medalhao.bronze.ft_pedidos")

    status_map = {
        "delivered": "entregue",
        "invoiced": "faturado",
        "shipped": "enviado",
        "processing": "em processamento",
        "unavailable": "indisponível",
        "canceled": "cancelado",
        "created": "criado",
        "approved": "aprovado"
    }

    map_args = []
    for key, value in status_map.items():
        map_args.append(F.lit(key))
        map_args.append(F.lit(value))
    
    map_expr = F.create_map(*map_args)

    df_enriched = df_bronze.withColumn(
        "tempo_entrega_dias", 
        F.datediff(F.col("order_delivered_customer_date"), F.col("order_purchase_timestamp"))
    ).withColumn(
        "tempo_entrega_estimado_dias",
        F.datediff(F.col("order_estimated_delivery_date"), F.col("order_purchase_timestamp"))
    )
    
    df_enriched = df_enriched.withColumn(
        "diferenca_entrega_dias", 
        F.col("tempo_entrega_dias") - F.col("tempo_entrega_estimado_dias")
    )

    entrega_prazo_logic = (
        F.when(F.col("order_delivered_customer_date").isNull(), "Não Entregue")
         .when(F.col("diferenca_entrega_dias") <= 0, "Sim")
         .otherwise("Não")
    )

    df_silver = df_enriched.select(
        F.col("order_id").alias("id_pedido"),
        F.col("customer_id").alias("id_consumidor"),
        
        F.coalesce(map_expr[F.col("order_status")], F.col("order_status")).alias("status"),
        
        F.col("order_purchase_timestamp").alias("pedido_compra_timestamp"),
        F.col("order_approved_at").alias("pedido_aprovado_timestamp"),
        F.col("order_delivered_carrier_date").alias("pedido_carregado_timestamp"),
        F.col("order_delivered_customer_date").alias("pedido_entregue_timestamp"),
        F.col("order_estimated_delivery_date").alias("pedido_estimativa_entrega_timestamp"),
        
        F.col("tempo_entrega_dias"),
        F.col("tempo_entrega_estimado_dias"),
        F.col("diferenca_entrega_dias"),
        entrega_prazo_logic.alias("entrega_no_prazo")
    )

    df_silver.write.mode("overwrite").saveAsTable("medalhao.silver.ft_pedidos")
    
    print("Tabela medalhao.silver.ft_pedidos criada.")

except Exception as e:
    print(f"Erro ao processar silver.ft_pedidos: {e}")

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import DecimalType

try:
    df_bronze = spark.table("medalhao.bronze.ft_itens_pedidos")

    df_silver = df_bronze.select(
        F.col("order_id").alias("id_pedido"),
        F.col("order_item_id").alias("id_item"),
        F.col("product_id").alias("id_produto"),
        F.col("seller_id").alias("id_vendedor"),
        
        F.col("price").cast(DecimalType(12, 2)).alias("preco_BRL"),
        F.col("freight_value").cast(DecimalType(12, 2)).alias("preco_frete")
    )

    df_silver.write.mode("overwrite").saveAsTable("medalhao.silver.ft_itens_pedidos")
    
    print("Tabela medalhao.silver.ft_itens_pedidos criada.")

except Exception as e:
    print(f"Erro ao processar silver.ft_itens_pedidos: {e}")

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import DecimalType

try:
    df_bronze = spark.table("medalhao.bronze.ft_pagamentos_pedidos")

    pagamento_map = {
        "credit_card": "Cartão de Crédito",
        "boleto": "Boleto",
        "voucher": "Voucher",
        "debit_card": "Cartão de Débito"
    }
    
    map_args = []
    for key, value in pagamento_map.items():
        map_args.append(F.lit(key))
        map_args.append(F.lit(value))
    
    map_expr = F.create_map(*map_args)

    df_silver = df_bronze.select(
        F.col("order_id").alias("id_pedido"),
        F.col("payment_sequential").alias("codigo_pagamento"),
        
        F.coalesce(map_expr[F.col("payment_type")], F.lit("Outro")).alias("forma_pagamento"),
        
        F.col("payment_installments").alias("parcelas"),
        F.col("payment_value").cast(DecimalType(12, 2)).alias("valor_pagamento")
    )

    df_silver.write.mode("overwrite").saveAsTable("medalhao.silver.ft_pagamentos")
    
    print("Tabela medalhao.silver.ft_pagamentos criada.")

except Exception as e:
    print(f"Erro ao processar silver.ft_pagamentos: {e}")

In [0]:
from pyspark.sql import functions as F

try:
    df_bronze = spark.table("medalhao.bronze.ft_avaliacoes_pedidos")
    
    linhas_antes = df_bronze.count()
    print(f"Linhas na Bronze: {linhas_antes}")

    df_com_datas_limpas = df_bronze.withColumn(
        "data_comentario_limpa",
        F.expr("try_cast(review_creation_date AS timestamp)")
    ).withColumn(
        "data_resposta_limpa",
        F.expr("try_cast(review_answer_timestamp AS timestamp)")
    )

    df_limpo = df_com_datas_limpas.filter(
        (F.col("order_id").isNotNull()) &
        (F.col("data_comentario_limpa").isNotNull()) &  
        (F.col("data_resposta_limpa").isNotNull()) &   
        (F.col("data_comentario_limpa") <= F.current_timestamp())
    )

    linhas_depois = df_limpo.count()
    linhas_removidas = linhas_antes - linhas_depois
    print(f"Linhas após limpeza: {linhas_depois}")
    print(f"DOCUMENTAÇÃO: {linhas_removidas} linhas removidas por ID nulo, datas nulas/corrompidas ou datas futuras.")

    df_silver = df_limpo.select(
        F.col("review_id").alias("id_avaliacao"),
        F.col("order_id").alias("id_pedido"),
        F.col("review_score").alias("avaliacao"),
        F.col("review_comment_title").alias("titulo_comentario"),
        F.col("review_comment_message").alias("comentario"),
        
        F.col("data_comentario_limpa").alias("data_comentario"),
        F.col("data_resposta_limpa").alias("data_resposta")
    )

    df_silver.write.mode("overwrite").saveAsTable("medalhao.silver.ft_avaliacoes_pedidos")
    
    print("Tabela medalhao.silver.ft_avaliacoes_pedidos criada.")

except Exception as e:
    print(f"Erro ao processar silver.ft_avaliacoes_pedidos: {e}")

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType

try:
    df_bronze = spark.table("medalhao.bronze.ft_produtos")

    df_silver = df_bronze.select(
        F.col("product_id").alias("id_produto"),
        F.col("product_category_name").alias("categoria_produto"),
        
        F.col("product_weight_g").cast(IntegerType()).alias("peso_produto_gramas"),
        F.col("product_length_cm").cast(IntegerType()).alias("comprimento_centimetros"),
        F.col("product_height_cm").cast(IntegerType()).alias("altura_centimetros"),
        F.col("product_width_cm").cast(IntegerType()).alias("largura_centimetros")
    )

    df_silver.write.mode("overwrite").saveAsTable("medalhao.silver.ft_produtos")
    
    print("Tabela medalhao.silver.ft_produtos criada.")

except Exception as e:
    print(f"Erro ao processar silver.ft_produtos: {e}")

In [0]:
from pyspark.sql import functions as F

try:
    df_bronze = spark.table("medalhao.bronze.ft_vendedores")

    df_silver = df_bronze.select(
        F.col("seller_id").alias("id_vendedor"),
        F.col("seller_zip_code_prefix").alias("prefixo_cep"),
        
        F.upper(F.col("seller_city")).alias("cidade"),
        F.upper(F.col("seller_state")).alias("estado")
    )

    df_silver.write.mode("overwrite").saveAsTable("medalhao.silver.ft_vendedores")
    
    print("Tabela medalhao.silver.ft_vendedores criada.")

except Exception as e:
    print(f"Erro ao processar silver.ft_vendedores: {e}")

In [0]:
from pyspark.sql import functions as F

try:
    df_bronze = spark.table("medalhao.bronze.dm_categoria_produtos_traducao")

    df_silver = df_bronze.select(
        F.col("product_category_name").alias("nome_produto_pt"),
        F.col("product_category_name_english").alias("nome_produto_en")
    )

    df_silver.write.mode("overwrite").saveAsTable("medalhao.silver.dm_categoria_produtos_traducao")
    
    print("Tabela medalhao.silver.dm_categoria_produtos_traducao criada.")

except Exception as e:
    print(f"Erro ao processar silver.dm_categoria_produtos_traducao: {e}")

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

try:
    df_bronze = spark.table("medalhao.bronze.dm_cotacao_dolar")

    df_cotacao_dias_uteis = df_bronze.select(
        F.to_date(F.col("dataHoraCotacao")).alias("data"),
        F.col("cotacaoCompra")
    ).distinct() 

    min_max_datas = df_cotacao_dias_uteis.agg(
        F.min("data").alias("data_min"),
        F.max("data").alias("data_max")
    ).first()
    
    data_min = min_max_datas["data_min"]
    data_max = min_max_datas["data_max"]

    df_calendario = spark.sql(f"SELECT explode(sequence(to_date('{data_min}'), to_date('{data_max}'), interval 1 day)) AS data")

    df_com_nulos = df_calendario.join(
        df_cotacao_dias_uteis,
        on="data",
        how="left"
    )

    window_spec = Window.orderBy("data").rowsBetween(Window.unboundedPreceding, Window.currentRow)
    
    df_preenchido = df_com_nulos.withColumn(
        "cotacao_preenchida",
        F.last(F.col("cotacaoCompra"), ignorenulls=True).over(window_spec)
    )

    df_silver = df_preenchido.select(
        F.col("cotacao_preenchida").alias("cotacao_dolar"), 
        F.col("data") 
    )
    
    df_silver.write.mode("overwrite").saveAsTable("medalhao.silver.dm_cotacao_dolar")
    
    print("Tabela medalhao.silver.dm_cotacao_dolar criada e preenchida.")

except Exception as e:
    print(f"Erro ao processar silver.dm_cotacao_dolar: {e}")

In [0]:
from pyspark.sql import functions as F

try:
    df_pedidos = spark.table("medalhao.silver.ft_pedidos")
    df_consumidores = spark.table("medalhao.silver.ft_consumidores")
    df_itens_pedidos = spark.table("medalhao.silver.ft_itens_pedidos")

    pedidos_orfos = df_pedidos.join(
        df_consumidores,
        on="id_consumidor",
        how="left_anti" 
    )
    
    count_pedidos_orfos = pedidos_orfos.count()
    print(f"Verificação 1: {count_pedidos_orfos} pedidos órfãos (sem consumidor) encontrados.")

    itens_orfos = df_itens_pedidos.join(
        df_pedidos,
        on="id_pedido",
        how="left_anti"
    )
    
    count_itens_orfos = itens_orfos.count()
    print(f"Verificação 2: {count_itens_orfos} itens de pedidos órfãos (sem pedido) encontrados.")

    
    if count_pedidos_orfos > 0:
        print(f"Removendo {count_pedidos_orfos} pedidos órfãos...")
        df_pedidos_limpos = df_pedidos.join(df_consumidores, on="id_consumidor", how="left_semi")
        
        df_pedidos_limpos.write.mode("overwrite").saveAsTable("medalhao.silver.ft_pedidos")
        print("Tabela medalhao.silver.ft_pedidos atualizada.")

    if count_itens_orfos > 0:
        print(f"Removendo {count_itens_orfos} itens órfãos...")
        df_itens_limpos = df_itens_pedidos.join(df_pedidos, on="id_pedido", how="left_semi")
        
        df_itens_limpos.write.mode("overwrite").saveAsTable("medalhao.silver.ft_itens_pedidos")
        print("Tabela medalhao.silver.ft_itens_pedidos atualizada.")

    if count_pedidos_orfos == 0 and count_itens_orfos == 0:
        print("Nenhum registro órfão encontrado.")

    print("Validações de integridade concluídas.")

except Exception as e:
    print(f"Erro durante a validação: {e}")

In [0]:
from pyspark.sql import functions as F

try:
    pedidos = spark.table("medalhao.silver.ft_pedidos")
    consumidores = spark.table("medalhao.silver.ft_consumidores")
    pagamentos = spark.table("medalhao.silver.ft_pagamentos")
    cotacao = spark.table("medalhao.silver.dm_cotacao_dolar")

    pagamentos_total_por_pedido = pagamentos.groupBy("id_pedido").agg(
        F.sum("valor_pagamento").alias("valor_total_pago_brl")
    )

    pedidos_com_data = pedidos.withColumn(
        "data_pedido", 
        F.to_date(F.col("pedido_compra_timestamp"))
    )

    df_joined = pedidos_com_data.join(
        consumidores,
        on="id_consumidor",
        how="left"
    ).join(
        pagamentos_total_por_pedido,
        on="id_pedido",
        how="left"
    ).join(
        cotacao,
        on=pedidos_com_data.data_pedido == cotacao.data,
        how="left"
    )

    df_silver_final = df_joined.select(
        F.col("data_pedido"),
        F.col("id_pedido"),
        F.col("id_consumidor"),
        F.col("status"),
        F.col("valor_total_pago_brl"),
        
        (F.round(F.col("valor_total_pago_brl") / F.col("cotacao_dolar"), 2)).alias("valor_total_pago_usd")
    )

    df_silver_final.write.mode("overwrite").saveAsTable("medalhao.silver.pedido_total")
    
    print("Tabela medalhao.silver.pedido_total criada.")
    
    print("Exemplo de dados da tabela final:")
    df_silver_final.show(5)

except Exception as e:
    print(f"Erro ao criar silver.pedido_total: {e}")